In [ ]:
## Useful Functions for Repeated Use
This notebook serves to input repeated functions for ease of script coding.

# useful import libraries
import pandas as pd
import numpy as np
import pickle
import joblib
import string
import re

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score
from pandas.tseries.offsets import MonthEnd

# models and inputs files
with_ess_features = True # condition if we want to use ESS features or not

# MACRO
file = open('outputs/US_data.pickle', "rb")
us_macro_df = pickle.load(file)

file = open('outputs/CN_data.pickle', "rb")
cn_macro_df = pickle.load(file)

# MODELS FOR SENTIMENT
file = open('outputs/tfidf_vect_tuned.pickle', "rb")
tfidf_vect = pickle.load(file)

if with_ess_features==True:
    
    file = open('outputs/xgb_monthly_combined_ESS_features.pickle', "rb") # under 04D Classification Model - latest xgb 220310
    xgb_monthly_combined = pickle.load(file)

    # ESS features (comment it out if using)
    filename = 'outputs/min_max_scaler_monthly_combined_ESS_features.save'
    min_max_scaler_monthly = joblib.load(filename)

    file = open('outputs/feature_select_dict_monthly_ESS_features.pickle', "rb")
    feature_select_dict_monthly = pickle.load(file)

### Based on Multicollinearity
Filtered based on manual and analysis on 04D script for classification model

# features to remove (manual basis) - for technical indicators, we will be using mid_window # 'percentile_rank_ratings_event_sentiment_score', 'percentile_rank_product_event_sentiment_score',

remove_features = ['model_score', 'rsi_5', 'rsi_50', 'evm_5', 'evm_50', 'bol_buy_5', 'bol_buy_50', 'bol_wband_5', 'bol_wband_50', 'adx_5', 'adx_50'] + ['percentile_rank_mergers_event_sentiment_score', 'percentile_rank_mergers_model_score', 'mergers_model_score', 'percentile_rank_product_model_score', 'product_model_score', 'percentile_rank_earnings_event_sentiment_score', 'percentile_rank_earnings_model_score', 'earnings_model_score',  'percentile_rank_ratings_model_score', 'ratings_model_score', 'percentile_rank_regulatory_event_sentiment_score', 'percentile_rank_regulatory_model_score', 'regulatory_model_score'] + ['percentile_rank_ratings_event_sentiment_score', 'percentile_rank_product_event_sentiment_score']

# training variables

X_clf = ["model_score", "event_sentiment_score", "vol",  "news_spikes_m", "returns", "gdp_growth_us", 'cpi_growth_us', 'unemployment_rate_us', "gdp_growth_cn", 'cpi_growth_cn', 'rsi_5', 'rsi_14', 'rsi_50', 'evm_5', 'evm_14', 'evm_50', 'bol_buy_5', 'bol_buy_14', 'bol_buy_50', 'bol_wband_5', 'bol_wband_14', 'bol_wband_50', 'adx_5', 'adx_14', 'adx_50', 'macd_12_26']  + ['mergers_event_sentiment_score', 'product_event_sentiment_score', 'earnings_event_sentiment_score', 'ratings_event_sentiment_score', 'regulatory_event_sentiment_score']  # X variables for classification
X_reg = ["model_score", "event_sentiment_score", "vol",  "news_spikes_m", "returns", "gdp_growth_us", 'cpi_growth_us', 'unemployment_rate_us', "gdp_growth_cn", 'cpi_growth_cn', 'rsi_5', 'rsi_14', 'rsi_50', 'evm_5', 'evm_14', 'evm_50', 'bol_buy_5', 'bol_buy_14', 'bol_buy_50', 'bol_wband_5', 'bol_wband_14', 'bol_wband_50', 'adx_5', 'adx_14', 'adx_50', 'macd_12_26']  + ['mergers_event_sentiment_score', 'product_event_sentiment_score', 'earnings_event_sentiment_score', 'ratings_event_sentiment_score', 'regulatory_event_sentiment_score']  # X variables for regression model

# after removing the features
X_clf = [i for i in X_clf if i not in remove_features]
X_reg = [i for i in X_reg if i not in remove_features]

y_clf = "returns_movement"    
y_reg = "returns_lead_1"

# if not using ESS features

if with_ess_features==False:
    additional_removal = ['mergers_event_sentiment_score',
    'product_event_sentiment_score',
    'earnings_event_sentiment_score',
    'ratings_event_sentiment_score',
    'regulatory_event_sentiment_score']

    X_clf = [i for i in X_clf if i not in additional_removal]

# script 03 general function

def aggregate_period(df, period, resample='mean'): # checked
    """
    Aggregating data based on required fields
    Period: D, W, M
    """
    if resample == 'mean':
        if period == 'D':
            df = df.resample('D').mean()

        elif period == 'W':
            df = df.resample('W-FRI').mean()

        elif period == 'M':
            df = df.resample('M').mean()

        else:
            raise Exception('Invalid timeframe from input')

    elif resample == 'last':
        if period == 'D':
            df = df.resample('D').last()

        elif period == 'W':
            df = df.resample('W-FRI').last()

        elif period == 'M':
            df = df.resample('M').last()

        else:
            raise Exception('Invalid timeframe from input')
            
    elif resample == 'sum':
        if period == 'D':
            df = df.resample('D').sum()

        elif period == 'W':
            df = df.resample('W-FRI').sum()

        elif period == 'M':
            df = df.resample('M').sum()

        else:
            raise Exception('Invalid timeframe from input')
            
    # ffill then bfill
    df = df.ffill(axis=0).bfill(axis=0)
    
    return df

def standardize_col_names(df, remove_punct=True): # checked
    """ 
    Converts all column names to lower case replacing
    whitespace of any length with a single underscore.
    Also, remove punctuations if included.

    """

    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

    for c in df.columns:

        c_mod = c.lower()

        if remove_punct:            
            c_mod = c_mod.translate(translator)

        c_mod = '_'.join(c_mod.split(' '))

        if c_mod[-1] == '_':
            c_mod = c_mod[:-1]

        c_mod = re.sub(r'\_+', '_', c_mod)

        df.rename({c: c_mod}, inplace=True, axis=1)

    return df

Note: Currently using event_sentiment_score to predict sentiment movement, similar to our model

def returns_movement(s):
    """
    Indicates the actual returns from historical data one period ahead
    """
    
    if (s['returns_lead_1'] > 0):
        return 1
    
    else:
        return 0
    

def financial_returns_movement(s):
    """
    Indicates the actual returns from historical data one period ahead
    """
    
    if (s['msci_financial_returns_lead_1'] > 0):
        return 1
    
    else:
        return 0
    
    
def msci_singapore_returns_movement(s):
    """
    Indicates the actual returns from historical data one period ahead
    """
    
    if (s['msci_singapore_returns_lead_1'] > 0):
        return 1
    
    else:
        return 0
    
def msci_china_returns_movement(s):
    """
    Indicates the actual returns from historical data one period ahead
    """
    
    if (s['msci_china_returns_lead_1'] > 0):
        return 1
    
    else:
        return 0

def performance(result, bank, period, ow_uw=False): # checked
    """
    Performance metrics based on the periodic time series
    """

    # filter the relevant banks first
    result = result[(result['sedol'] == bank)]
    result.sort_index(inplace=True)

    # cumulative returns - for both bm and pm
    return_bm_cum = result.iloc[-1]['return_bm_cum']-1
    return_mp_cum = result.iloc[-1]['return_mp_cum']-1

    if period == "M":
        n_months= len(result.index)
        n_years=n_months/12
        
    elif period == "W":
        n_weeks= len(result.index)
        n_years=n_weeks/52
    
    else:
        raise Exception('Period window is not compatible. Please check if the time series is either weekly or monthly.')
    
    print(f"{bank}: {n_years} years of datasets")
    # auc=roc_auc_score(result['returns_movement'], result['position'])
    f1score=f1_score(result['returns_movement'], result['position'], average='weighted')
    precision=precision_score(result['returns_movement'], result['position'], average='weighted')

    ## trade lot and positive
    if ow_uw: # overweight underweight winrate calculation based on excess return
        winrate=(np.array(result[(result['position_lag_1']!=0)]['excess_return_mp'])>0).mean()
            
    else:
        winrate=(np.array(result[(result['position_lag_1']!=0)]['return_mp'])>1).mean()

    anl_rtn_bm =(result[['return_bm','return_mp']].cumprod()**(1/n_years)).iloc[-1,0]
    anl_rtn_mp=(result[['return_bm','return_mp']].cumprod()**(1/n_years)).iloc[-1,1]
    anl_ex_rtn= anl_rtn_mp-anl_rtn_bm
    
    turnover=abs(result['position'].diff()).sum()/n_years/2
    
    ## sharpe ratio
    result['year']=pd.to_datetime(result.index).year
    result_year=result.groupby('year').agg('last')
    return_bm_y=(result_year['return_bm_cum']/result_year['return_bm_cum'].shift(1)-1).dropna() # series, not single value
    return_mp_y=(result_year['return_mp_cum']/result_year['return_mp_cum'].shift(1)-1).dropna()# series, not single value
    return_excess_y=return_mp_y-return_bm_y # portfolio - benchmark
    volatility_bm=return_bm_y.std() # to display as output performance
    volatility_mp=return_mp_y.std() # to display as output performance
    avg_ex_rtn=return_excess_y.mean()

    if return_excess_y.std() == 0:
        sharpe_ratio = "nil"
    else:
        sharpe_ratio=avg_ex_rtn/return_excess_y.std()
            
    ## max_drawdown: buy and hold
    max_drawdown_bm=0
    for i in range(result.shape[0]):
        temp=result['return_bm_cum'][i:].min()/result['return_bm_cum'][i]-1
        
        if temp<max_drawdown_bm:
            max_drawdown_bm=temp
    
    ## max_drawdown: sentiment
    max_drawdown_mp=0
    for i in range(result.shape[0]):
        temp=result['return_mp_cum'][i:].min()/result['return_mp_cum'][i]-1
        
        if temp<max_drawdown_mp:
            max_drawdown_mp=temp

    return [anl_rtn_bm,anl_rtn_mp,anl_ex_rtn,winrate,turnover,volatility_bm,volatility_mp,max_drawdown_bm,max_drawdown_mp,return_bm_cum,return_mp_cum,sharpe_ratio,f1score,precision]

def performance_breakdown(result, bank, period, ow_uw=False): # checked
    """
    Detailed breakdown of performance metrics based on yearly basis
    """
    
    ldf = []
    
    for y in range(2019, 2023):
        
        # filtering the specific date
        train_df = result[(result.index >= pd.to_datetime(f'{y}-01-01')) & (result.index <= pd.to_datetime(f'{y}-12-31'))]
        train_df.reset_index(inplace=True, drop=True)
        train_df.sort_index(inplace=True) # sort based on date
        
        # cumulative returns - for both bm and pm
        return_bm_cum = (train_df['return_bm']).cumprod().iloc[-1] -1
        return_mp_cum = (train_df['return_mp']).cumprod().iloc[-1] -1
        
        
        if period == "M":
            n_months= len(train_df.index)
            n_years=n_months/12

        elif period == "W":
            n_weeks= len(train_df.index)
            n_years=n_weeks/52
            
        else:
            raise Exception('Period window is not compatible. Please check if the time series is either weekly or monthly.')

        # auc=roc_auc_score(train_df['returns_movement'], train_df['position'])
        f1score=f1_score(train_df['returns_movement'], train_df['position'], average='weighted')
        precision=precision_score(train_df['returns_movement'], train_df['position'], average='weighted')

        if ow_uw: # overweight underweight winrate calculation based on excess return
            winrate=(np.array(train_df[(train_df['position_lag_1']!=0)]['excess_return_mp'])>0).mean()
            
        else:
            winrate=(np.array(train_df[(train_df['position_lag_1']!=0)]['return_mp'])>1).mean()

        anl_rtn_bm=(train_df[['return_bm','return_mp']].cumprod()**(1/n_years)).iloc[-1,0]
        anl_rtn_mp=(train_df[['return_bm','return_mp']].cumprod()**(1/n_years)).iloc[-1,1]
        anl_ex_rtn= anl_rtn_mp-anl_rtn_bm

        turnover=abs(train_df['position'].diff()).sum()/n_years/2

        ## sharpe ratio
        train_df['year']=pd.to_datetime(train_df.index).year
        result_year=train_df # no need to agg as df is in yearly freq
        return_bm_y=(result_year['return_bm_cum']/result_year['return_bm_cum'].shift(1)-1).dropna() # series, not single value
        return_mp_y=(result_year['return_mp_cum']/result_year['return_mp_cum'].shift(1)-1).dropna()# series, not single value
        return_excess_y=return_mp_y-return_bm_y # portfolio - benchmark
        volatility_bm=return_bm_y.std()
        volatility_mp=return_mp_y.std()
        avg_ex_rtn=return_excess_y.mean()
        
        if return_excess_y.std() == 0: # for extreme values checking
            sharpe_ratio="nil"
        else:
            sharpe_ratio=avg_ex_rtn/return_excess_y.std()

        ## max_drawdown: buy and hold
        max_drawdown_bm=0
        
        for i in range(train_df.shape[0]):
            temp=train_df['return_bm_cum'][i:].min()/train_df['return_bm_cum'][i]-1

            if temp<max_drawdown_bm:
                max_drawdown_bm=temp

        ## max_drawdown: sentiment
        max_drawdown_mp=0
        for i in range(train_df.shape[0]):
            temp=train_df['return_mp_cum'][i:].min()/train_df['return_mp_cum'][i]-1

            if temp<max_drawdown_mp:
                max_drawdown_mp=temp
        
        df1 = pd.DataFrame(index=['Annualised Return (Buy Hold)','Annualised Return (Sentiment)','Annualised Excess Return', 'Winrate', "Annualised Turnover", 'Volatility (Buy Hold)', 'Volatility (Sentiment)', 'Max Drawdown (Buy Hold)', 'Max Drawdown (Sentiment)',"Cumulative Return (Buy Hold)", "Cumulative Return (Sentiment)", 'Sharpe Ratio', 'F1 Score', "Precision"])
        df1[f"{y}"] = [anl_rtn_bm,anl_rtn_mp,anl_ex_rtn,winrate,turnover,volatility_bm,volatility_mp,max_drawdown_bm,max_drawdown_mp,return_bm_cum,return_mp_cum,sharpe_ratio,f1score,precision]

        ldf.append(df1)
    
    
    df = pd.concat(ldf, axis=1)
    
    return df

def perf_format(performance_df, benchmark=False): # checked
    """
    Formatting the performance dataframe into desired precision and format
    """
    if benchmark==True:
        # converting certain rows into percentage
        performance.index_name = 'Metrics'
        performance_df = performance_df.round(decimals=5)
        performance_df.loc['Annualised Return (Benchmark)'] = ((performance_df.loc['Annualised Return (Benchmark)']-1).apply('{:.02%}'.format))
        performance_df.loc['Annualised Return (Sentiment)'] = ((performance_df.loc['Annualised Return (Sentiment)']-1).apply('{:.02%}'.format))
        performance_df.loc['Annualised Excess Return'] = ((performance_df.loc['Annualised Excess Return']).apply('{:.02%}'.format))
        performance_df.loc['Winrate'] = ((performance_df.loc['Winrate']).apply('{:.02%}'.format))
        performance_df.loc['Volatility (Benchmark)'] = ((performance_df.loc['Volatility (Benchmark)']).apply('{:.02%}'.format))
        performance_df.loc['Volatility (Sentiment)'] = ((performance_df.loc['Volatility (Sentiment)']).apply('{:.02%}'.format))
        performance_df.loc['Max Drawdown (Benchmark)'] = ((performance_df.loc['Max Drawdown (Benchmark)']).apply('{:.02%}'.format))
        performance_df.loc['Max Drawdown (Sentiment)'] = ((performance_df.loc['Max Drawdown (Sentiment)']).apply('{:.02%}'.format))

        
    else:
        # converting certain rows into percentage
        performance.index_name = 'Metrics'
        performance_df = performance_df.round(decimals=5)
        performance_df.loc['Annualised Return (Buy Hold)'] = ((performance_df.loc['Annualised Return (Buy Hold)']-1).apply('{:.02%}'.format))
        performance_df.loc['Annualised Return (Sentiment)'] = ((performance_df.loc['Annualised Return (Sentiment)']-1).apply('{:.02%}'.format))
        performance_df.loc['Annualised Excess Return'] = ((performance_df.loc['Annualised Excess Return']).apply('{:.02%}'.format))
        performance_df.loc['Winrate'] = ((performance_df.loc['Winrate']).apply('{:.02%}'.format))
        performance_df.loc['Volatility (Buy Hold)'] = ((performance_df.loc['Volatility (Buy Hold)']).apply('{:.02%}'.format))
        performance_df.loc['Volatility (Sentiment)'] = ((performance_df.loc['Volatility (Sentiment)']).apply('{:.02%}'.format))
        performance_df.loc['Max Drawdown (Buy Hold)'] = ((performance_df.loc['Max Drawdown (Buy Hold)']).apply('{:.02%}'.format))
        performance_df.loc['Max Drawdown (Sentiment)'] = ((performance_df.loc['Max Drawdown (Sentiment)']).apply('{:.02%}'.format))

    return performance_df

# for classification
def _train_test_split(df, entity_list, test_date): # checked
    """
    Allow iterative train test split for the dataframe
    """
    
    df.sort_index(axis=0, ascending=True, inplace=True)

    # train test split based on date
    X = df[X_clf]
    X_train = X[X.index < test_date]
    X_test = X[X.index >= test_date]

    y = df[f'{y_clf}']
    y_train = y[y.index < test_date]
    y_test = y[y.index >= test_date]

    return X_train, y_train, X_test, y_test

def model_performance(models, model_names, feature_select_dict): # checked
    """
    Machine learning related performance metrics, specifically for classification models
    """
    df = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 score', 'ROC AUC'])

    for n, model in enumerate(models):
        name = model_names[n]
        y_pred = model.predict(X_test[feature_select_dict[name]])

        acc = metrics.accuracy_score(y_test, y_pred)
        prec = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        roc_auc = metrics.roc_auc_score(y_test, y_pred)

        #append row to df
        df = df.append(
            {
                'Model' : name,
                'Accuracy': acc,
                'Precision': prec,
                'Recall': recall,
                'F1 score': f1,
                'ROC AUC': roc_auc
            }, ignore_index = True)

    return df.set_index('Model').transpose()

def prediction_processing(df, model, sentiment, freq, quantile=0.7, ess_threshold=0.2, returns_threshold=0.5, news_spikes_threshold=2.0): # checked
    """
    Data manipulation for model prediction, default process to run the model
    """
    
    # handling of frequency prediction (ie, weekly or monthly)
    if freq == "W":
        min_max_scaler = min_max_scaler_weekly
        feature_select_dict = feature_select_dict_weekly
    
    elif freq == "M":
        min_max_scaler = min_max_scaler_monthly
        feature_select_dict = feature_select_dict_monthly
    
    # scaling and transformation 
    X = df[X_clf]
    X[X_clf] = min_max_scaler.transform(X[X_clf])

    # Model Prediction
    result = model.predict(X[feature_select_dict])
    pred_result = model.predict_proba(X[feature_select_dict])
    df['returns_predict'] = result
    df['returns_predict_proba'] = pred_result[:,1] # get the prob of 1
    
    if sentiment:
    
        df['position'] = df.apply(lambda r: model_positions(r, True, True, quantile, ess_threshold, returns_threshold, news_spikes_threshold), axis=1)
        
    else: # use model as prediction
        
        df['position'] = df.apply(lambda r: model_positions(r, False, True, quantile, ess_threshold, returns_threshold, news_spikes_threshold), axis=1)
        
    df['position_lag_1'] = df['position'].shift(1) # use previous months position to calculate performance metrics

    # Get the weekly log returns (assuming i make a decision on a daily basis)
    df['return_bm'] = df['returns']+1 # shift by 1 period (ie, curr - prev row)
    df['return_bm_cum'] = (df['return_bm']).cumprod()

    df['return_mp'] = df.apply(lambda x: x['return_bm'] if x['position_lag_1'] == 1 else 1.0, axis=1) # finding the previous position and multiply it by current month returns
    df['return_mp_cum'] = (df['return_mp']).cumprod()
    
    # excess return mp for winrate calculation
    df['excess_return_mp'] = df['return_mp'] - df['return_bm']
    
    # creating strategy directions: both movement is for one period forward
    df['sentiment_movement'] = df['event_sentiment_score'].apply(lambda x: 1 if x > 0 else 0)
    df['returns_movement'] = df.apply(lambda x: returns_movement(x), axis=1)
    
    return df

def prediction_processing_financials(df, hk_financials_sedol_list, use_predict_proba, quantile, returns_threshold, news_spikes_threshold):
    """
    Model positioning for msci financials
    """
    df = aggregate_financials(df, hk_financials_sedol_list, use_predict_proba)
        
    df['position'] = df.apply(lambda r: financial_positions(r, True, quantile, returns_threshold, news_spikes_threshold), axis=1)        
    df['position_lag_1'] = df['position'].shift(1) # use previous months position to calculate performance metrics
    
    # Get the monthly log returns (assuming i make a decision on a daily basis)
    df['return_bm'] = df['msci_financial_returns']+1 # shift by 1 period (ie, curr - prev row)
    df['return_bm_cum'] = (df['return_bm']).cumprod()
    
    df['return_mp'] = df.apply(lambda x: x['return_bm'] if x['position_lag_1'] == 1 else 1.0, axis=1) # finding the previous position and multiply it by current month returns
    df['return_mp_cum'] = (df['return_mp']).cumprod()
    
    # excess return mp for winrate calculation
    df['excess_return_mp'] = df['return_mp'] - df['return_bm']
    
    # creating strategy directions: movement is one period forward
    df['returns_movement'] = df.apply(lambda x: financial_returns_movement(x), axis=1)

    return df

def aggregate_financials(df, hk_financials_sedol_list, use_predict_proba=True):
    """
    Aggregate the variables based on the weighted average of the individual financial stocks composition, using predict_proba
    """
    # weighted returns based on AI/ML, using predict_proba
    df['tot_fin_weights'] = 1 - df['others'] # total overall financial weights
    df['returns_predict'] = calculate_returns_predict(df, hk_financials_sedol_list, use_predict_proba)
    
    # percentile_rank_ratings_event_sentiment_score, percentile_rank_product_event_sentiment_score, percentile_rank_regulatory_event_sentiment_score, news_spikes_m
    df['regulatory_event_sentiment_score'] = calculate_ess_group(df, hk_financials_sedol_list, "regulatory_event_sentiment_score")
    df['product_event_sentiment_score'] = calculate_ess_group(df, hk_financials_sedol_list, "product_event_sentiment_score")
    df['ratings_event_sentiment_score'] = calculate_ess_group(df, hk_financials_sedol_list, "ratings_event_sentiment_score")
    df['news_spikes_m'] = calculate_ess_group(df, hk_financials_sedol_list, "news_spikes_m")
    df['macd_12_26'] = calculate_ess_group(df, hk_financials_sedol_list, "macd_12_26")

    # percentile ranking allocation    
    
    event_grouping_list = ['regulatory', 'product', 'ratings', 'macd_12_26']
    
    # filter out the important group list
    for e in event_grouping_list:
        if e == "macd_12_26":
            df[f"percentile_rank_{e}"] = df[f'{e}'].rank(pct=True)
            continue
        # percentile rank for the col
        df[f"percentile_rank_{e}_event_sentiment_score"] = df[f'{e}_event_sentiment_score'].rank(pct=True)
    
    required_cols = [ # relevant columns for the remaining codes
        'returns_predict',
        'msci_china_price',
        'msci_china_returns',
        'msci_china_returns_lead_1',
        'msci_financial_price',
        'msci_financial_returns', 
        'msci_financial_returns_lead_1',
        'tot_fin_weights',
        'others',
        'percentile_rank_regulatory_event_sentiment_score', 
        'percentile_rank_product_event_sentiment_score', 
        'percentile_rank_ratings_event_sentiment_score',
        'percentile_rank_macd_12_26',
        'news_spikes_m'
    ]
    
    df = df[required_cols]
    
    return df

def calculate_returns_predict(df, hk_financials_sedol_list, use_predict_proba):
    """
    To calculate the returns predict proba based on aggregated weights and individual firm predict proba
    """
    ldf = []
    if use_predict_proba:
        
        for sedol in hk_financials_sedol_list:
            df[f'{sedol}_weighted_returns_predict_proba'] = df[f'{sedol}_returns_predict_proba']*df[sedol]/df['total_hk_fin_weights']
            ldf.append(df[[f'{sedol}_weighted_returns_predict_proba']])
    
    else:
        
        for sedol in hk_financials_sedol_list:
            df[f'{sedol}_weighted_returns_predict'] = df[f'{sedol}_returns_predict']*df[sedol]/df['total_hk_fin_weights']
            ldf.append(df[[f'{sedol}_weighted_returns_predict']])
    
    new_df = pd.concat(ldf, axis=1)

    return new_df.sum(axis=1).to_list()

def calculate_ess_group(df, hk_financials_sedol_list, col_name):
    """
    To calculate the returns predict proba based on aggregated weights and individual firm predict proba
    """
    
    ldf = []
    for sedol in hk_financials_sedol_list:
        df[f'{sedol}_weighted_{col_name}'] = df[f'{sedol}_{col_name}']*df[sedol]/df['total_hk_fin_weights']
        ldf.append(df[[f'{sedol}_weighted_{col_name}']])
    
    new_df = pd.concat(ldf, axis=1)
    
    return new_df.sum(axis=1).to_list()

# script 05 general functions

def model_positions(row, sentiment=False, ess_x_spikes=False, quantile=0.7, ess_threshold=0.2, returns_threshold=0.5, news_spikes_threshold=2.0): # checked
    """
    Returns position as 1, 0 for Buy and Hold respectively 
    based on the predicted price value and threshold
    FYI: predicting forward returns
    """

    if sentiment: # using current ess to make positioning for future returns outcome
        
        if row['event_sentiment_score'] > ess_threshold: # based on optimal threshold calculated based on 05D monthly datasets
            output = 1
        else:
            output = 0
    
    else: # following the score based on predicted returns
    
        if row['returns_predict_proba'] > returns_threshold: # same as using returns_predict > 0
            output = 1

        else:
            if row['percentile_rank_regulatory_event_sentiment_score'] > quantile: # rule based regime override
                output = 1
            else:
                output = 0
        
        # ultimate check on buy strategy:
        if row['percentile_rank_product_event_sentiment_score'] > quantile: # most positive sentiment score 

            output = 0 # due to the strong negative correlation for financials
        
        if ess_x_spikes == True:
            
            if output == 1: # one final check on buying in with wrong signal (minimise potential drawdown)
                          
                if row['news_spikes_m'] > news_spikes_threshold: # increase confidence that negative drawdown higher       
                    output = 0
        
#         if row["percentile_rank_macd_12_26"] > quantile: # above the historical 75th percentile, strong conviction to buy as momentum serve as a conviction
            
#             output = 1
    
    return output

# # script 05 general functions

# def model_positions(row, sentiment=False, ess_x_spikes=False, quantile=0.7, ess_threshold=0.2, returns_threshold=0.5, news_spikes_threshold=2.0): # checked
#     """
#     Returns position as 1, 0 for Buy and Hold respectively 
#     based on the predicted price value and threshold
#     FYI: predicting forward returns
#     """

#     if sentiment: # using current ess to make positioning for future returns outcome
        
#         if row['event_sentiment_score'] > ess_threshold: # based on optimal threshold calculated based on 05D monthly datasets
#             output = 1
#         else:
#             output = 0
    
#     else: # following the score based on predicted returns
    
#         if row['returns_predict_proba'] > returns_threshold: # same as using returns_predict > 0
#             output = 1

#         else:
#             if row['percentile_rank_ratings_event_sentiment_score'] > quantile or row['percentile_rank_product_event_sentiment_score'] > quantile: # rule based regime override
#                 output = 1
#             else:
#                 output = 0
    
#         # ultimate check on buy strategy:
#         if row['percentile_rank_mergers_event_sentiment_score'] > quantile: # most positive sentiment score 

#             output = 0 # due to the strong negative correlation for financials
        
#         if ess_x_spikes == True:
            
#             if output == 1: # one final check on buying in with wrong signal (minimise potential drawdown)
                          
#                 if row['news_spikes_m'] > news_spikes_threshold: # increase confidence that negative drawdown higher       
#                     output = 0
                
#     return output

def financial_positions(row, ess_x_spikes=False, quantile=0.7, returns_threshold=0.5, news_spikes_threshold=2.0):
    """
    Returns position as 1, 0 for Buy and Hold respectively 
    based on the combined predicted returns value
    """

    # weighted returns based on AI/ML, using predict_proba
    if row['returns_predict'] > returns_threshold: # buy and hold (dont make a move for 1)
        output = 1
    
    else:
        output = 0
            
    return output

# def financial_positions(row, ess_x_spikes=False, quantile=0.7, returns_threshold=0.5, news_spikes_threshold=2.0):
#     """
#     Returns position as 1, 0 for Buy and Hold respectively 
#     based on the combined predicted returns value
#     """

#     # weighted returns based on AI/ML, using predict_proba
#     if row['returns_predict'] > returns_threshold: # buy and hold (dont make a move for 1)
#         output = 1
    
#     # rules-based regime on the allocation aggregate with the individual stocks weightng to come out with a final aggregated allocation
#     else:
#         if row['percentile_rank_ratings_event_sentiment_score'] > quantile or row['percentile_rank_regulatory_event_sentiment_score'] > quantile: # rule based regime override
#             output = 1
#         else:
#             output = 0
    
#     # ultimate check on buy strategy:
#     if row['percentile_rank_product_event_sentiment_score'] > quantile: # most positive sentiment score 

#         output = 0 # due to the strong negative correlation for financials

# #     if ess_x_spikes == True:

# #         if output == 1: # one final check on buying in with wrong signal (minimise potential drawdown)

# #             if row['news_spikes_m'] > news_spikes_threshold: # increase confidence that negative drawdown higher       
# #                 output = 0


#     return output

# else:
#     if row['percentile_rank_regulatory_event_sentiment_score'] > quantile: # rule based regime override
#         output = 1
#     else:
#         output = 0

# # ultimate check on buy strategy:
# if row['percentile_rank_product_event_sentiment_score'] > quantile: # most positive sentiment score 

#     output = 0 # due to the strong negative correlation for financials

# if ess_x_spikes == True:

#     if output == 1: # one final check on buying in with wrong signal (minimise potential drawdown)

#         if row['news_spikes_m'] > news_spikes_threshold: # increase confidence that negative drawdown higher       
#             output = 0

# if row["percentile_rank_macd_12_26"] > quantile: # above the historical 75th percentile, strong conviction to buy as momentum serve as a conviction

#     output = 1
    

# # 1.52 percentage

# def financial_positions(row, ess_x_spikes=False, quantile=0.7, returns_threshold=0.5, news_spikes_threshold=2.0):
#     """
#     Returns position as 1, 0 for Buy and Hold respectively 
#     based on the combined predicted returns value
#     """

#     # weighted returns based on AI/ML, using predict_proba
#     if row['returns_predict'] > returns_threshold: # buy and hold (dont make a move for 1)
#         output = 1
    
#     # rules-based regime on the allocation aggregate with the individual stocks weightng to come out with a final aggregated allocation
#     else:
#         if row['percentile_rank_regulatory_event_sentiment_score'] > quantile: # rule based regime overrideride
#             output = 1
#         else:
#             output = 0
    
# #     # ultimate check on buy strategy:
# #     if row['percentile_rank_product_event_sentiment_score'] > quantile: # most positive sentiment score 

# #         output = 0 # due to the strong negative correlation for financials

# #     if ess_x_spikes == True:

# #         if output == 1: # one final check on buying in with wrong signal (minimise potential drawdown)

# #             if row['news_spikes_m'] > news_spikes_threshold: # increase confidence that negative drawdown higher       
# #                 output = 0


#     return output

def weight_allocation(df, weight=0.05):
    """
    Assign weights allocation based on threshold, add in additional weights based on model signals
    Current: 0.05 +/-
    """
    
    df['return_bm'] = df['msci_china_returns']+1
    df['return_bm_cum'] = (df['return_bm']).cumprod()
    
    df['return_mp'] = df.apply(lambda x: x['msci_financial_returns'] * (x['tot_fin_weights'] + weight) \
    + x['msci_other_returns'] * (x['others'] - weight) if x['position_lag_1'] == 1 \
    else x['msci_financial_returns'] * (x['tot_fin_weights'] - weight) \
    + x['msci_other_returns'] * (x['others'] + weight), axis=1)+1

#     # The following is for 100% accurate forecasting, ie, knowing the future (just for presentation)
#     df['return_mp'] = df.apply(lambda x: x['msci_financial_returns'] * (x['tot_fin_weights'] + weight) \
#     + x['msci_other_returns'] * (x['others'] - weight) if x['msci_financial_returns'] > x['msci_other_returns'] \
#     else x['msci_financial_returns'] * (x['tot_fin_weights'] - weight) \
#     + x['msci_other_returns'] * (x['others'] + weight), axis=1)+1
#     df['position'] = df.apply(lambda x: 1 if x['msci_financial_returns'] > x['msci_other_returns'] else 0, axis=1)
    
    df['return_mp_cum'] = (df['return_mp']).cumprod()
    
    # excess return mp for winrate calculation
    df['excess_return_mp'] = df['return_mp'] - df['return_bm']

    # creating strategy directions
    df['returns_movement'] = df.apply(lambda x: msci_china_returns_movement(x), axis=1)
    df['sedol'] = 'msci_china'                      
    
    return df

def optimize_weight_distribution(df, max_weight=0.15):
    """
    Find optimal figure for weight distribution
    Eg, +/- 5%??
    """
    max_weight = int(max_weight*1000) # converting weight to whole number
    best_performance = 0.05
    best_excess_return = 0.0
    
    for i in range(5, max_weight+5, 5): # use integer: 0.005 to 0.01
        i = i*0.001
        print(f'===== Checking weights: {i*100}% =====')
        df = weight_allocation(df, i)
        performance_list = performance(df, 'msci_china', 'M', True) # set overweight underweight to true
        print()

        if performance_list[2] > best_excess_return:
            
            best_performance = i
            best_excess_return = performance_list[2] # 3rd element is the annualized excess return (optimize this value)
    
    return best_performance, best_excess_return

def optimize_quantile(df, bank_name, model, sentiment, frequency, quantile=0.95, ess_threshold=0.2, returns_threshold=0.5, news_spikes_threshold=2.0): # checked
    """
    Find optimal quantile for best rules-based signal based on best predicted returns
    Customise for individual companies
    Percentile optimization starts from 80th percentile onwards
    """
    quantile = int(quantile*100) 

    best_performance_values = []
    best_excess_return = 0.0
    
    for i in range(80, quantile+5, 5): # use integer: 0.8 to max_quantile
        print(f'===== Checking quantile: {i*0.01}% =====')
        print()
        i = i*0.01
        
        for j in range(15, 35, 5): # for ess threshold, 0.15 to 0.3
            j = j*0.01
            
            for k in range(40, 65, 5): # for returns threshold, 0.4 to 0.6
                k = k*0.01
                
                for l in range(20, 40, 5): # for news spikes threshold, +2.0 to +3.5
                    l = l*0.1

                    df =  prediction_processing(df, model, sentiment, frequency, i, j, k, l) # sentiment strategy as True

                    performance_list = performance(df, bank_name, 'M')


                    if performance_list[2] > best_excess_return:

                        best_performance_values = [i, j, k, l]
                        best_excess_return = performance_list[2] # 3rd element is the annualized excess return (optimize this value)
    
    if len(best_performance_values) == 0: # empty list
        
        best_performance_values = [0.75, 0.2, 0.5, 2.0] # default value for negative excess returns cases
        print("=== Negative Excess Returns Detected ===")
        
    return best_performance_values, best_excess_return

# cleaned version of optimize financial quantile (matching with zero rule based regime in the financial model position method)

def optimize_financial_quantile(df, bank_name, hk_financials_sedol_list, quantile=0.95): # checked
    """
    Find optimal quantile for best rules-based signal
    Customise for individual companies
    Percentile optimization starts from 80th percentile onwards
    """
    quantile = int(quantile*100) 

    best_performance_values = []
    best_excess_return = -1
    
    for i in range(30, 75, 5): # for returns threshold, 0.3 to 0.7)
        i = i*0.01
        final_df = prediction_processing_financials(df, hk_financials_sedol_list, True, 0.8, i, 2.0)
        final_df['sedol'] = bank_name
        performance_list = performance(final_df, bank_name, 'M')

        if performance_list[2] > best_excess_return:

            best_performance_values = [0.8, i, 2.0]
            best_excess_return = performance_list[2] # 3rd element is the annualized excess return (optimize this value)

    if len(best_performance_values) == 0: # empty list
        
        best_performance_values = [0.8, 0.6, 2.0] # default value for negative excess returns cases
        print("=== Negative Excess Returns Greater than -100% Detected ===")
        
    return best_performance_values, best_excess_return

# def optimize_financial_quantile(df, bank_name, hk_financials_sedol_list, quantile=0.95): # checked
#     """
#     Find optimal quantile for best rules-based signal
#     Customise for individual companies
#     Percentile optimization starts from 80th percentile onwards
#     """
#     quantile = int(quantile*100) 

#     best_performance_values = []
#     best_excess_return = -1
    
#     for i in range(80, quantile+5, 5): # use integer: 0.8 to max_quantile
#         print(f'===== Checking quantile: {i}% =====')
#         print()
#         i = i*0.01
        
#         for j in range(30, 75, 5): # for returns threshold, 0.3 to 0.7
#             j = j*0.01
            
#             for k in range(20, 40, 5): # for news spikes threshold, +2.0 to +3.0
#                 k = k*0.1
#                 final_df = prediction_processing_financials(df, hk_financials_sedol_list, True, i, j, k)
#                 final_df['sedol'] = bank_name
#                 performance_list = performance(final_df, bank_name, 'M')

#                 if performance_list[2] > best_excess_return:

#                     best_performance_values = [i, j, k]
#                     best_excess_return = performance_list[2] # 3rd element is the annualized excess return (optimize this value)
    
#     if len(best_performance_values) == 0: # empty list
        
#         best_performance_values = [0.75, 0.5, 2.0] # default value for negative excess returns cases
#         print("=== Negative Excess Returns Greater than -100% Detected ===")
        
#     return best_performance_values, best_excess_return

# def optimize_financial_quantile(df, bank_name, hk_financials_sedol_list, quantile=0.95): # checked
#     """
#     Find optimal quantile for best rules-based signal
#     Customise for individual companies
#     Percentile optimization starts from 80th percentile onwards
#     """
#     quantile = int(quantile*100) 

#     best_performance_values = []
#     best_excess_return = -1
    
#     for i in range(80, quantile+5, 5): # use integer: 0.8 to max_quantile
#         print(f'===== Checking quantile: {i}% =====')
#         print()
#         i = i*0.01
        
#         for j in range(30, 75, 5): # for returns threshold, 0.3 to 0.7
#             j = j*0.01
            
#             for k in range(20, 40, 5): # for news spikes threshold, +2.0 to +3.0
#                 k = k*0.1
#                 final_df = prediction_processing_financials(df, hk_financials_sedol_list, True, i, j, k)
#                 final_df['sedol'] = bank_name

#                 # filtering only relevant columns for weight allocation for msci financials
#                 filter_cols = [
#                     'position',
#                     'position_lag_1',
#                     'msci_china_price',
#                     'msci_china_returns',
#                     'msci_china_returns_lead_1', # for performance measure
#                     'msci_financial_price',
#                     'msci_financial_returns',
#                     'msci_financial_returns_lead_1',
#                     "tot_fin_weights",
#                     "others",
#                     "returns_predict", 
#                 ]

#                 final_df = final_df[filter_cols]
#                 final_df['msci_other_returns'] = ((final_df['msci_china_returns'] - final_df['msci_financial_returns']* final_df['tot_fin_weights']/100) / (final_df['others']/100))/100
    
#                 # Get best model based on performance in excess return from OW/UW strategy
#                 final_df = weight_allocation(final_df, 0.1) # using 0.1
#                 performance_list = performance(final_df, 'msci_china', 'M', True) # set overweight underweight to true

#                 if performance_list[2] >= best_excess_return:

#                     best_performance_values = [i, j, k]
#                     best_excess_return = performance_list[2] # 3rd element is the annualized excess return (optimize this value)
    
#     if len(best_performance_values) == 0: # empty list
        
#         best_performance_values = [0.75, 0.5, 2.0] # default value for negative excess returns cases
#         print("=== Negative Excess Returns Greater than -100% Detected ===")
        
#     return best_performance_values, best_excess_return

# def optimize_quantile(df, bank_name, model, sentiment, frequency, max_quantile=0.95):
#     """
#     Find optimal quantile for best rules-based signal
#     Customise for individual companies
#     Percentile optimization starts from 80th percentile onwards
#     """
#     max_quantile = int(max_quantile*1000) 
#     best_performance_quantile = 0.75
#     best_excess_return = 0.0
    
#     for i in range(800, max_quantile+5, 5): # use integer: 0.8 to max_quantile
        
#         i = i*0.001
#         print(f'===== Checking quantile: {i*100}% =====')
#         df =  prediction_processing(df, model, sentiment, frequency, i) # sentiment strategy as True

#         performance_list = performance(df, bank_name, 'M')
#         print()

#         if performance_list[2] > best_excess_return:
            
#             best_performance_quantile = i
#             best_excess_return = performance_list[2] # 3rd element is the annualized excess return (optimize this value)
    
#     return best_performance_quantile, best_excess_return

### Event Feature Engineering For Event Group
- used in 03_Production_Stage_New_Feature_Technical

def event_feature_engineering(df): # checked
    """
    Generate event group features for rule-based regime, generalize based on the entire dataset
    Note: Percentile Ranking is based off entire duration of the dataframe (not daily, weekly, monthly)
    """
    event_grouping_dict = {
        'mergers': ['acquisitions-mergers', 'partnerships'],
        'product': ['marketing', 'products-services'],
        'earnings': ['dividends', 'earnings', 'revenues', 'credit-ratings'],
        'ratings': ['analyst-ratings', 'stock-prices', 'price-targets'],
        'regulatory': ['bankruptcy', 'regulatory', 'legal']    
    }
    
    # filter out the important group list
    for e, groups in event_grouping_dict.items():
        
        temp_df = df[df['group'].isin(groups)][['event_sentiment_score', 'model_score']]
        temp_df.rename(columns={'event_sentiment_score': f"{e}_event_sentiment_score",'model_score': f"{e}_model_score" }, inplace=True)
        
        df = df.merge(temp_df, left_index=True, right_index=True, how='left')
        
        # Fill NA values as 0
        df[f"{e}_event_sentiment_score"] = df[f"{e}_event_sentiment_score"].fillna(0)
        df[f"{e}_model_score"] = df[f"{e}_model_score"].fillna(0)
        
    return df

def event_percentile_rank(df): # checked
    """
    Create new percentile feature and study the correlation between the returns movement of the price
    Note: only use it when aggregated into periodic frequency, eg, based on daily or weekly or monthly
    """
    event_grouping_dict = {
        'mergers': ['acquisitions-mergers', 'partnerships'],
        'product': ['marketing', 'products-services'],
        'earnings': ['dividends', 'earnings', 'revenues', 'credit-ratings'],
        'ratings': ['analyst-ratings', 'stock-prices', 'price-targets'],
        'regulatory': ['bankruptcy', 'regulatory', 'legal']    
    }
    
    # filter out the important group list
    for e, groups in event_grouping_dict.items():
        
        # percentile rank for the col
        df[f"percentile_rank_{e}_event_sentiment_score"] = df[f'{e}_event_sentiment_score'].rank(pct=True)
        df[f"percentile_rank_{e}_model_score"] = df[f'{e}_model_score'].rank(pct=True)
     
    return df

def macd_feature_engineering(df, s):
    """
    To create macd percentile rank per sedol
    """
    # do an inner merge with original dataframe based on index and sedol
    df = df[df['sedol'] == i]
    df['percentile_rank_macd_12_26'] = df['macd_12_26'].rank(pct=True)
    
    return df